# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [39]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [40]:
# your code here
from functools import reduce
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import sys
sys.executable


# Connection Data
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'orders'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
# Engine & Query
engine = create_engine(connection_string)
query = 'SELECT * FROM orders'
# Database Request
orders = pd.read_sql(query,engine)



In [41]:
orders.head(1)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.3


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


In [42]:
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [45]:
# PART 1
rev_by_cust = orders.groupby("CustomerID")["amount_spent"].sum().to_frame()
rev_by_cust

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
12352,2506.04
12353,89.00
12354,1079.40
12355,459.40


In [ ]:
#rev_by_cust['quantile'] = pd.qcut(rev_by_cust['amount_spent'], 100, labels=False)
rev_by_cust

# Emanuele's way didnt work. 
conditions = [
    (rev_by_cust['quantile'] >= 75 ) & (rev_by_cust['quantile'] <= 95),
    (rev_by_cust['quantile'] > 95)]
choices = ['Preferred', 'VIP']
rev_by_cust['Status'] = np.select(conditions, choices, default='Regular')
print(rev_by_cust)

In [18]:
rev_by_cust.to_csv(r'rev_by_custVIP.csv')

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [95]:
# your code here
#custbycountry = orders[['Country','CustomerID']]
#custbycountry = custbycountry.drop_duplicates(subset=['Country','CustomerID'])
country_status = pd.merge(custbycountry, rev_by_cust, on='CustomerID', how='left') # CHECK THE JOIN
country_status

#country_status.groupby(['CustomerID', 'Status']).agg(['count'])
#country_status.groupby(['Status', 'Country']).agg(['count'])

country_status.groupby(['Status', 'Country'])['CustomerID'].count()# .sort('CustomerID')

#country_status.groupby(['Status', 'Country'])['CustomerID'].count().filter(lambda x: len(x) > 1)
#  df = df[['STNAME','CTYNAME']].groupby(['STNAME']).agg(['count']).sort(['count'])

Status     Country             
Preferred  Australia                  3
           Austria                    3
           Belgium                   12
           Canada                     1
           Channel Islands            3
           Cyprus                     4
           Denmark                    2
           EIRE                       1
           Finland                    4
           France                    21
           Germany                   31
           Greece                     1
           Iceland                    1
           Israel                     2
           Italy                      5
           Japan                      2
           Lebanon                    1
           Malta                      1
           Norway                     6
           Poland                     1
           Portugal                   5
           Spain                      7
           Sweden                     1
           Switzerland                7
        

In [116]:
vip_count_cnt = country_status[country_status["Status"] == 'VIP'].groupby(['Status', 'Country'])['CustomerID'].count()
vip_count_cnt.nlargest
    #country_status.groupby(['Status', 'Country'])['CustomerID'].count()# .sort('CustomerID')

<bound method Series.nlargest of Status  Country        
VIP     Australia            1
        Channel Islands      1
        Denmark              1
        EIRE                 2
        Finland              1
        France               8
        Germany              8
        Japan                2
        Netherlands          1
        Norway               1
        Portugal             2
        Singapore            1
        Spain                2
        Sweden               1
        Switzerland          2
        United Kingdom     140
Name: CustomerID, dtype: int64>

In [120]:
vip_count_cnt = country_status[country_status["Status"] == 'VIP'].groupby(['Status', 'Country'])['CustomerID'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(9)
vip_count_cnt

,Status,Country,count
15,VIP,United Kingdom,140
5,VIP,France,8
6,VIP,Germany,8
3,VIP,EIRE,2
7,VIP,Japan,2
10,VIP,Portugal,2
12,VIP,Spain,2
14,VIP,Switzerland,2
0,VIP,Australia,1


In [119]:
preferred_count_cnt = country_status[country_status["Status"] == 'Preferred'].groupby(['Status', 'Country'])['CustomerID'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)
preferred_count_cnt

,Status,Country,count
24,Preferred,United Kingdom,792
10,Preferred,Germany,31
9,Preferred,France,21
2,Preferred,Belgium,12
23,Preferred,Switzerland,7


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [123]:
# your code here

preferred_count_cnt2 = country_status[country_status["Status"] != 'Regular'].groupby([ 'Country'])['CustomerID'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)
preferred_count_cnt2.head(1)

,Country,count
26,United Kingdom,932
